In [1]:
from pymilvus import MilvusClient
mv_client = MilvusClient("course_info.db")

In [71]:
if mv_client.has_collection(collection_name="course_info"):
    mv_client.drop_collection(collection_name="course_info")
mv_client.create_collection(
    collection_name="course_info",
    dimension=768,  # The vectors we will use in this demo has 768 dimensions
)

In [72]:
import json 
with open('document.json') as f_in: 
    docs_raw = json.load(f_in)
documents = []
for course_dict in docs_raw : 
    for doc in course_dict['documents'] : 
        doc['course'] = course_dict['course']
        documents.append(doc)

In [73]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-mpnet-base-v2")

/home/nhutpham/anaconda3/envs/N2/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [74]:
import hashlib

def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [10]:
#created the dense vector using the pre-trained model
operations = []
for i, doc in enumerate(documents):
    doc["id"] = i
    doc["vector"] = model.encode(doc["text"]).tolist()
    operations.append(doc)

In [43]:
prompt_template = """
You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record.

The record:

section: {section}
question: {question}
answer: {text}

* NOTE: PROVIDE THE OUTPUT IN PARSABLE JSON JUST OUTPUT TEXT WITHOUT USING CODE BLOCKS

["question1", "question2", ..., "question5"]
""".strip()

In [44]:
import google.generativeai as genai
import os 
from dotenv import load_dotenv

load_dotenv()
genai.configure(api_key=os.environ["GEMINI_API_KEY"])
model = genai.GenerativeModel(model_name=("gemini-1.5-flash-002"))

In [45]:
def generate_questions(prompt): 
    answer = model.generate_content(prompt)
    return answer.text

In [46]:
from tqdm.auto import tqdm
results = {}

In [47]:
import time 
for doc in tqdm(documents): 
    doc_id = doc['id']
    prompt = prompt_template.format(**doc)
    questions = generate_questions(prompt)
    time.sleep(5)
    results[doc_id] = questions

  0%|          | 0/948 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
with open('documents-with-ids.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)

In [49]:
results

{0: '["What happens if I can\'t make the first Office Hours on January 15th?", "Is the Google Calendar invite the only way to get reminders about the course?", "If I register after January 15th, will I still be able to access course materials?", "What information do I need to provide when I register on DataTalks.Club\'s Slack?", "Besides the Telegram channel, are there other ways to receive course updates?"]\n',
 1: '["Is there a specific version of Python required for the course?", "Are the prerequisites strictly enforced, or is it possible to succeed in the course with some missing skills?", "How much prior experience with SQL is expected, according to the prerequisites?", "What is the recommended way to check if I meet all the prerequisites before enrolling?", "The prerequisites mention GitHub; does that mean I\'ll need to be comfortable with Git and version control from day one?"]\n',
 2: '["If I join late, will I have access to all the course materials from the beginning?", "What 

In [52]:
parsed_resulst = {}

for doc_id, json_questions in results.items():
    parsed_resulst[doc_id] = json.loads(json_questions)

In [53]:
doc_index = {d['id']: d for d in documents}

In [54]:
final_results = []

for doc_id, questions in parsed_resulst.items():
    course = doc_index[doc_id]['course']
    for q in questions:
        final_results.append((q, course, doc_id))

In [55]:
import pandas as pd
df = pd.DataFrame(final_results, columns=['question', 'course', 'document'])
df.to_csv('ground-truth-data.csv', index=False)

In [61]:
print(df['question'])

0     What happens if I can't make the first Office ...
1     Is the Google Calendar invite the only way to ...
2     If I register after January 15th, will I still...
3     What information do I need to provide when I r...
4     Besides the Telegram channel, are there other ...
5     Is there a specific version of Python required...
6     Are the prerequisites strictly enforced, or is...
7     How much prior experience with SQL is expected...
8     What is the recommended way to check if I meet...
9     The prerequisites mention GitHub; does that me...
10    If I join late, will I have access to all the ...
11    What happens if I miss the deadlines for the h...
12    Are there any penalties for joining the course...
13    Will late submissions of homeworks be accepted...
14    Besides the final project deadlines, are there...
15    If registration isn't required, why is it even...
16    What happens if I register but don't complete ...
17    Is there a deadline for registering, even 

In [7]:
print("Data has", len(documents), "entities, each with fields: ", documents[0].keys())
print("Vector dim:", len(documents[0]["vector"]))

Data has 948 entities, each with fields:  dict_keys(['text', 'section', 'question', 'course', 'id', 'vector'])
Vector dim: 768


In [8]:
res = mv_client.insert(collection_name="course_info", data=documents)

print(res)

RPC error: [insert_rows], <DataNotMatchException: (code=1, message=The Input data type is inconsistent with defined schema, {id} field should be a int64, but got a {<class 'str'>} instead.)>, <Time:{'RPC start': '2024-10-07 04:10:50.408053', 'RPC error': '2024-10-07 04:10:50.410195'}>


DataNotMatchException: <DataNotMatchException: (code=1, message=The Input data type is inconsistent with defined schema, {id} field should be a int64, but got a {<class 'str'>} instead.)>

In [36]:
def elastic_search(query) : 
    query_vectors = [model.encode(query)]
    res = mv_client.search(
        collection_name="course_info",  
        data=query_vectors,  
        filter="course == 'data-engineering-zoomcamp'",
        limit=5,  
        output_fields=["text", "question", "section", "course"],  
    )
    return res[0]

In [42]:
def build_prompt(query, search_result) : 
    prompt_template = """
        You're a course teaching assistant. Answer the QUESTION based on the the CONTEXT. 
        Use only the facts from the CONTEXT when answering the QUESTION. 
        If the CONTEXT doesn't contain the answer, output NONE 

        QUESTION: {question}

        CONTEXT: 
        {context}
        """.strip()
    context = ""

    for doc in search_result: 
        entity = doc['entity']
        context = context + f"section: {entity['section']}\nquestion: {entity['question']}\nanswer: {entity['text']}\n\n"
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt
    

In [44]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',)
def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [45]:
def rag(query): 
    results = elastic_search(query)
    prompt = build_prompt(query, results)
    answer = llm(prompt)
    return answer

In [48]:
print(rag('I just discovered the course. Can i still join ?'))

Based on the CONTEXT provided:

It appears that since you're just now discovering or registering for this Data Engineering Bootcamp hosted by Coursera through DATAtalkSquare, there is still time to join if registration hasn't been processed yet as courses typically have a period during which they remain open. The context does not explicitly state whether it’s still possible to enroll after certain deadlines or policies are in place; therefore from this information alone I would say YES, you could likely register even though the course has started since registration usually opens before the start date of 15th Jan 2024 at 17h00 as per Question: "Course - When will the course start?" Here's your answer assuming a general policy for educational institutions allowing prospective students to still enroll until official class materials or sessions begin.

As there is no specific mention of deadlines, policies on joining after starting in other sections of questions within these provided conte